# compas_fea2 simple script approach

This example covers the analysis of a simple 2D frame structure generated completely from scratch. 

### NOTE: This example is in SI (m, kg, s)

In [1]:
from pathlib import Path
from pprint import pprint

from compas_fea2.backends.abaqus import Model
from compas_fea2.backends.abaqus import Part
from compas_fea2.backends.abaqus import Node
from compas_fea2.backends.abaqus import ElasticIsotropic
from compas_fea2.backends.abaqus import BoxSection
from compas_fea2.backends.abaqus import BeamElement

from compas_fea2.backends.abaqus import Problem
from compas_fea2.backends.abaqus import GeneralStaticStep

from compas_fea2.backends.abaqus import Results

from compas_fea2 import TEMP

## Model generation

The fist step (of an FEA) is the generation of the analysis `model`. 
A model is always composed by:

1) geometry: nodes, connectivity among nodes, parts in which nodes and elements are groups, etc.
The geometry can either be created from scratch (as in this example) or be imported from other datastructures (networks, meshes, etc).

2) properties:  materials, sections, elements, interactions, etc. Properties are applied to the geometry to simulate the mechanical behaviour. 

In a compas workflow, geometry and properties are provided by the compas_package that creates the structure to analyse. 


In [2]:
model = Model(name='structural_model')

print(model)


compas_fea2 Model object
------------------------
author          : None
bcs             : {}
constraints     : {}
contacts        : {}
description     : None
dtype           : 'compas_fea2.backends/Model'
interactions    : {}
materials       : {}
name            : 'structural_model'
parts           : {}
parts_groups    : {}
sections        : {}
surfaces        : {}



### Parts

`Parts` are subregions of the model that can be considered to be independent from each other. However, similar parts of a model can still be modelled as different `Parts`. 

A good example of a `Part` could be a brick. 

In [3]:
model.add_part(Part(name='frame'))
print(model.parts['frame'])


compas_fea2 Part object
-----------------------
dtype           : 'compas_fea2.backends/Part'
elements        : {}
groups          : {}
materials       : {}
name            : 'frame'
nodes           : []
releases        : {}
sections        : {}



Nodes can be added to the `Part` or directly to the `Model` by specifying the name of the `Part` to which they belong.
In this example, we want to create a simple rectangular frame with a node on each side of the frame every 100mm.

In [4]:
for x in range(0, 11, 1):
    model.add_node(Node(xyz=[x, 0.0, 0.0]), part='frame')
for y in range(1, 6, 1):
    model.add_node(Node(xyz=[x, y, 0.0]), part='frame')
for x in range(9, -1, -1):
    model.add_node(Node(xyz=[x, y, 0.0]), part='frame')
for y in range(4, 0, -1):
    model.add_node(Node(xyz=[x, y, 0.0]), part='frame')

print(model.parts['frame'].nodes[5])


compas_fea2 Node object
-----------------------
dtype           : 'compas_fea2.backends/Node'
ex              : None
ey              : None
ez              : None
gkey            : '5.000,0.000,0.000'
key             : 5
mass            : None
name            : 'n-5'
x               : 5.0
xyz             : [5.0, 0.0, 0.0]
y               : 0.0
z               : 0.0



We can now add the material properties we plan to use in our analysis. These material will then be available in the 'Model' and can be used in the definition of the structural sections.

In [5]:
# Define materials
model.add_material(ElasticIsotropic(name='mat_A', E=29*10*9, v=0.17, p=2500))
print(model.materials['mat_A'])


compas_fea2 ElasticIsotropic object
-----------------------------------
E               : 2610
G               : 1115.3846153846155
dtype           : 'compas_fea2.backends/ElasticIsotropic'
name            : 'mat_A'
p               : 2500
unilateral      : None
v               : 0.17



`compas_fea2` uses the same convention as Abaqus where the materials are assigned to the section and not to the elements (sections are assigned to the elements).
In the definition of a structural section you can specify a particular material by passing the material name. The structural properties are computed automatically accordingly to the type of section chosen (`BoxSection` in this case)

In [6]:
# Define sections
model.add_section(BoxSection(name='section_A', material='mat_A', b=0.20, h=0.80, t=0.01))

print(model.sections['section_A'])


compas_fea2 BoxSection object
-----------------------------
A               : 0.019600000000000006
Avx             : 0.0
Avy             : 0.0
Ixx             : 0.0014150533333333343
Ixy             : 0.0
Iyy             : 0.0001542533333333333
J               : 0.00045979612244897967
dtype           : 'compas_fea2.backends/BoxSection'
g0              : 0.0
guid            : UUID('f1d08d81-7521-4501-821f-453871664cd4')
gw              : 0.0
material        : ElasticIsotropic(mat_A)
name            : 'section_A'
properties      : [0.2, 0.8, 0.01, 0.01, 0.01, 0.01]
stype           : 'box'



Structural elements are defined by their type, connectivity and associated section. In this case we use a `BeamElement`, so we need to specify the 2 nodes it is connected to, with the 'BoxSection' defined before.
Elements can be added to the `model` one by one or as a list. 

In [7]:
# Generate elements between nodes
elements = []
for e in range(len(model.parts['frame'].nodes)-1):
    elements.append(BeamElement(connectivity=[e, e+1], section='section_A'))
elements.append(BeamElement(connectivity=[len(model.parts['frame'].nodes)-1, 0], section='section_A'))
model.add_elements(elements=elements, part='frame')

print(model.parts['frame'].elements[0])



compas_fea2 BeamElement object
------------------------------
axes            : None
connectivity    : [0, 1]
connectivity_key : '0_1'
dtype           : 'compas_fea2.backends/BeamElement'
eltype          : 'B31'
key             : 0
name            : 'None'
orientation     : [0.0, 0.0, -1.0]
section         : BoxSection(section_A)
thermal         : None



## Initial Boundary Conditions

In [8]:
# Assign boundary conditions to the node stes
model.add_fix_bc('fixed', 'frame', [0])
model.add_rollerXZ_bc('roller', 'frame', [10])

print(model.bcs)

{'frame': {FixedBC(fixed): [0], RollerBCXZ(roller): [10]}}


In [9]:
model.summary()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
compas_fea2 Model: structural_model
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

description: N/A
author: N/A

Parts
-----
frame
    # of nodes: 30
    # of elements: 30

Materials
---------
ElasticIsotropic(mat_A)

Sections
--------
BoxSection(section_A)

Interactions
------------


Constraints
-----------


Boundary Conditions
-------------------
frame: {FixedBC(fixed): [0], RollerBCXZ(roller): [10]}



'\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\ncompas_fea2 Model: structural_model\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\ndescription: N/A\nauthor: N/A\n\nParts\n-----\nframe\n    # of nodes: 30\n    # of elements: 30\n\nMaterials\n---------\nElasticIsotropic(mat_A)\n\nSections\n--------\nBoxSection(section_A)\n\nInteractions\n------------\n\n\nConstraints\n-----------\n\n\nBoundary Conditions\n-------------------\nframe: {FixedBC(fixed): [0], RollerBCXZ(roller): [10]}\n'

## Problem definition
The second part of the FEA is to define the `Problem` to be solved. To define a `Problem` you need a `Model`, boundary conditions (`bcs`), `laods`, field and history `outputs` and analysis 'steps'.  

In [10]:

# Create the Problem object
# Create the Problem object
problem = Problem(name='test_structure', model=model)

# Define the analysis step
problem.add_step(GeneralStaticStep(name='step-1'))

# Assign a point load to the node set
problem.add_point_load(name='pload', step='step-1', part='frame', where=5, y=-10000)

# Review
problem.summary()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
compas_fea2 Problem: test_structure
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

description: Problem for Model(structural_model)
author: N/A

Steps (in order of application)
-------------------------------
step-1: GeneralStaticStep




'\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\ncompas_fea2 Problem: test_structure\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\ndescription: Problem for Model(structural_model)\nauthor: N/A\n\nSteps (in order of application)\n-------------------------------\nstep-1: GeneralStaticStep\n\n'

Once the `Probelm` is set, you can solve it running the analysis. You need to specify the location of the analysis files generated by Abaqus.
The output log for now is limited to the one automatically generated by Abaqus, but the plan is to expand it to have more info about the status of the analysis, possible warnings or errors, etc. 

In [11]:
# # Solve the problem
problem.analyse(path=Path(TEMP).joinpath(problem.name))

***** Input File(test_structure) generated in: c:\code\myrepos\from_compas\fea2\temp\test_structure\test_structure.inp *****

Analysis initiated from SIMULIA established products
Abaqus JOB test_structure
Abaqus 2021
Begin Analysis Input File Processor
2/6/2022 3:36:50 PM
Run pre.exe
2/6/2022 3:36:54 PM
End Analysis Input File Processor
Begin Abaqus/Standard Analysis
2/6/2022 3:36:54 PM
Run standard.exe
2/6/2022 3:36:57 PM
End Abaqus/Standard Analysis
Begin SIM Wrap-up
2/6/2022 3:36:58 PM
Run SMASimUtility.exe
2/6/2022 3:36:59 PM
End SIM Wrap-up
Abaqus JOB test_structure COMPLETED

Abaqus License Manager checked out the following licenses:
Abaqus/Standard checked out 5 tokens from Flexnet server lic-abaqus.ethz.ch.
<299 out of 320 licenses remain available>.

***** Analysis successful - analysis time : 13.40173053741455 s *****


## Results Visualization

...not there yet... :)

In [12]:
##### --------------------- POSTPROCESS RESULTS -------------------------- #####
results = Results.from_problem(problem, fields=['u'])
pprint(results.nodal)


Abaqus License Manager checked out the following license:
"cae_teaching" from Flexnet server lic-abaqus.ethz.ch
<37 out of 40 licenses remain available>.


***** Data extracted from Abaqus .odb file : 5.398 s *****

***** Data stored successfully : 0.008 s *****

{'step-1': {'um': {0: 0.0,
                   1: 2504.41748046875,
                   2: 7618.1865234375,
                   3: 13730.3486328125,
                   4: 19229.890625,
                   5: 22505.802734375,
                   6: 21284.533203125,
                   7: 17326.123046875,
                   8: 11728.1884765625,
                   9: 5589.2314453125,
                   10: 210.8977813720703,
                   11: 3382.796630859375,
                   12: 5616.32763671875,
                   13: 6781.89990234375,
                   14: 7171.71826171875,
                   15: 7077.98779296875,
                   16: 7061.42041015625,
                   17: 7062.0419921875,
                   18: 7088.